In [14]:
import numpy
import ast 
import urllib
import scipy.optimize
import csv
import random
from sklearn import linear_model
import gzip
from collections import defaultdict

In [3]:
dataset = []
with open("RAW_interactions.csv", encoding="utf8") as f:
    next(f)  
    for l in f:
        fields = l.strip().split(",")  # Remove trailing newline and split by commas
        dataset.append(fields)

In [4]:
dataset[0]
#user_id, recipe_id, date, rating, review

['38094',
 '40893',
 '2003-02-17',
 '4',
 'Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt.  Used low fat sour cream.  Thanks.']

In [5]:
recipe_ratings = {}

for row in dataset:
    # Validate that row has enough fields and fields are not empty
    if len(row) >= 4 and row[1].strip() and row[3].strip():
        recipe_id = row[1].strip()  # Clean whitespace
        rating = row[3].strip()
        
        # Additional check: Ensure 'rating' is numeric, if applicable
        if rating.isdigit():
            # Add rating to the dictionary
            if recipe_id not in recipe_ratings:
                recipe_ratings[recipe_id] = []
            recipe_ratings[recipe_id].append(int(rating))

# Example output
#for recipe_id, ratings in recipe_ratings.items():
#    print(f"Recipe ID: {recipe_id}, Ratings: {ratings}")

In [6]:
dataset_features = []
with open("RAW_recipes.csv", encoding="utf8") as f:
    next(f)  
    for l in f:
        fields = l.strip().split(",")  # Remove trailing newline and split by commas
        dataset_features.append(fields)
#name, id, minutes, contributor_id, date, tags, nutrition, number of steps, steps, description, ingredients, n_ingredinets

In [15]:
recipe_nutrition = {}

with open("RAW_recipes.csv", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header

    for row in reader:
        if len(row) >= 7:  # Ensure the row has enough fields
            recipe_id = row[1].strip()  # Recipe ID (assumed second column)
            nutrition_data = row[6].strip()  # Nutrition field (assumed seventh column)

            try:
                # Safely parse the nutrition field
                nutrition_list = ast.literal_eval(nutrition_data)

                # Ensure the parsed data is a list and has at least 7 elements
                if isinstance(nutrition_list, list) and len(nutrition_list) >= 7:
                    total_fat = nutrition_list[1]  # Total fat (PDV)
                    sugar = nutrition_list[2]      # Sugar (PDV)
                    saturated_fat = nutrition_list[5]  # Saturated fat (PDV)

                    # Validate that the values are non-negative numbers
                    if all(isinstance(x, (int, float)) and x >= 0 for x in [total_fat, sugar, saturated_fat]):
                        recipe_nutrition[recipe_id] = {
                            "total_fat": total_fat,
                            "sugar": sugar,
                            "saturated_fat": saturated_fat
                        }
            except (ValueError, SyntaxError, TypeError):
                # Skip rows with invalid nutrition data
                continue

# Example output
#for recipe_id, nutrition in recipe_nutrition.items():
#   print(f"Recipe ID: {recipe_id}, Nutrition: {nutrition}")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




Recipe ID: 413236, Nutrition: {'total_fat': 38.0, 'sugar': 8.0, 'saturated_fat': 56.0}
Recipe ID: 87945, Nutrition: {'total_fat': 37.0, 'sugar': 3.0, 'saturated_fat': 72.0}
Recipe ID: 241398, Nutrition: {'total_fat': 13.0, 'sugar': 27.0, 'saturated_fat': 7.0}
Recipe ID: 114472, Nutrition: {'total_fat': 248.0, 'sugar': 27.0, 'saturated_fat': 228.0}
Recipe ID: 107696, Nutrition: {'total_fat': 24.0, 'sugar': 14.0, 'saturated_fat': 11.0}
Recipe ID: 57659, Nutrition: {'total_fat': 32.0, 'sugar': 7.0, 'saturated_fat': 14.0}
Recipe ID: 182534, Nutrition: {'total_fat': 31.0, 'sugar': 8.0, 'saturated_fat': 62.0}
Recipe ID: 349385, Nutrition: {'total_fat': 7.0, 'sugar': 307.0, 'saturated_fat': 7.0}
Recipe ID: 177323, Nutrition: {'total_fat': 12.0, 'sugar': 12.0, 'saturated_fat': 6.0}
Recipe ID: 331457, Nutrition: {'total_fat': 1.0, 'sugar': 11.0, 'saturated_fat': 0.0}
Recipe ID: 407418, Nutrition: {'total_fat': 60.0, 'sugar': 24.0, 'saturated_fat': 81.0}
Recipe ID: 311614, Nutrition: {'total_fa